(regression-diagnostics)=
# Regression diagnostics and visualisations

## Introduction

In this chapter, you'll see some diagnostics and visualisations that are closely tied to regressions.

Most of this chapter will rely on the amazing [**statsmodels**](https://www.statsmodels.org/stable/index.html) package, including for some of the examples. Some of the material in this chapter follows [Grant McDermott](https://grantmcdermott.com/)'s excellent notes and the [Library of Statistical Translation](https://lost-stats.github.io/).

## Imports

Let's import some of the packages we'll be using:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
from lets_plot import *

LetsPlot.setup_html()

# Set seed for random numbers
seed_for_prng = 78557
prng = np.random.default_rng(
    seed_for_prng
)  # prng=probabilistic random number generator

In [ ]:
import matplotlib_inline.backend_inline

# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

# Set max rows displayed for readability
pd.set_option("display.max_rows", 6)

## Regression plots

**statsmodels** has a number of built-in plotting methods to help you understand how well your regression is capturing the relationships you're looking for. Let's see a few examples of these using **statsmodels** built-in Statewide Crime data set:



In [ ]:
crime_data = sm.datasets.statecrime.load_pandas()
print(sm.datasets.statecrime.NOTE)

First, let's look at a Q-Q plot to get a sense of how the variables are distributed. This uses **scipy**'s stats module. The default distribution is normal but you can use any that **scipy** supports.

In [ ]:
st.probplot(crime_data.data["murder"], dist="norm", plot=plt);

Clearly, this is not quite normal and there are some serious outliers in the tails.

Let's run take a look at the unconditional relationship we're interested in: how murder depends on high school graduation. We'll use [**lets-plot**](https://lets-plot.org/)'s `geom_smooth()` to do this but bear in mind it will only run a linear model of `'murder ~ hs_grad'` and ignore the other covariates.

In [ ]:
p = (
    ggplot(crime_data.data, aes(y="murder", x="hs_grad"))
    + geom_point()
    + geom_smooth(method="lm")
)
p.show()

We can take into account those other factors by using a partial regression plot that asks what does $\mathbb{E}(y|X)$ look like as a function of $\mathbb{E}(x_i|X)$? (Use `obs_labels=False` to remove data point labels.)

In [ ]:
with plt.rc_context({"font.size": 5}):
    sm.graphics.plot_partregress(
        endog="murder",
        exog_i="hs_grad",
        exog_others=["urban", "poverty", "single"],
        data=crime_data.data,
        obs_labels=True,
    )
    plt.show()

At this point, the results of the regression are useful context.

In [ ]:
results_crime = smf.ols(
    "murder ~ hs_grad + urban + poverty + single", data=crime_data.data
).fit()
print(results_crime.summary())

Putting the multicollinearity problems to one side, we see that the relationship shown in the partial regression plot is also implied by the coefficient on `hs_grad` in the regression table.

We can also look at an in-depth summary of one exogenous regressor and its relationship to the outcome variable. Each of these types of regression diagnostic are available individually, or for all regressors at once, too. The first panel is the chart we did with **lets-plot** rendered differently (and, one could argue, more informatively). Most of the plots below are self-explanatory except for the third one, the CCPR (Component-Component plus Residual) plot. This provides a way to judge the effect of one regressor on the response variable by taking into account the effects of the other independent variables. 

In [ ]:
fig = plt.figure(figsize=(8, 6), dpi=150)

sm.graphics.plot_regress_exog(results_crime, "hs_grad", fig=fig)
plt.show()

**statsmodels** can also produce influence plots of the 'externally studentised' residuals vs. the leverage of each observation as measured by the so-called hat matrix $X(X^{\;\prime}X)^{-1}X^{\;\prime}$ (because it puts the 'hat' on $y$). Externally studentised residuals are residuals that are scaled by their standard deviation. High leverage points could exert an undue influence over the regression line, but only if the predicted $y$ values of a regression that was fit with them excluded was quite different. In the example below, DC is having a big influence.

In [ ]:
with plt.rc_context({"font.size": 6}):
    sm.graphics.influence_plot(results_crime)

Finally, it's nice to be able to see plots of our coefficients along with their standard errors. There isn't a built-in **statsmodels** option for this, but happily it's easy to extract the results of regressions in a sensible format. Using the `results` object from earlier, and excluding the intercept, we can get the coefficients from `results.params[1:]` and the associated errors from `results.bse[1:]`.

In [ ]:
# Put the results into a dataframe with Name, Coefficient, Error
res_df = (
    pd.concat([results_crime.params[1:], results_crime.bse[1:]], axis=1)
    .reset_index()
    .rename(columns={"index": "Name", 0: "Coefficient", 1: "Error"})
)
res_df["ymax"] = res_df["Coefficient"] + res_df["Error"]
res_df["ymin"] = res_df["Coefficient"] - res_df["Error"]
# Plot the coefficient values and their errors
(
    ggplot(res_df)
    + geom_point(aes("Name", "Coefficient"))
    + geom_errorbar(aes(x="Name", ymin="ymin", ymax="ymax"))
).show()

## Binscatter

We're going to follow the excellent example in this [blog post](https://towardsdatascience.com/goodbye-scatterplot-welcome-binned-scatterplot-a928f67413e4) by [Matteo Courthoud](https://www.linkedin.com/in/matteo-courthoud/).

First we'll generate some fake data:

In [ ]:
def dgp_marketplace(N=10_000):
    # Does the firm sells only online?
    online = prng.binomial(1, 0.5, N)
    # How many products does the firm have
    products = 1 + prng.poisson(1, N)
    # What is the age of the firm
    t = prng.exponential(0.5 * products, N)
    # Sales
    sales = 1e3 * prng.exponential(
        products
        + np.maximum(
            (1 + 0.3 * products + 4 * online) * t - 0.5 * (1 + 6 * online) * t**2, 0
        ),
        N,
    )
    # Generate the dataframe
    df = pd.DataFrame(
        {"age": t, "sales": sales, "online": online, "products": products}
    )
    return df

In [ ]:
sales = dgp_marketplace(N=10_000)
sales.head()

We have pulled down information on 10,000 firms. For each firm we know:

 - `age`: the age of the firm
 - `sales`: the monthly sales from last month
 - `online`: whether the firm is only active online
 - `products`: the number of products that the firm offers

Suppose we are interested in understanding the relationship between age and sales. What is the life-cycle of sales?

Let’s start with a simple scatterplot of sales vs age.

In [ ]:
(ggplot(sales, aes(x="age", y="sales")) + geom_point(color="blue", alpha=0.4)).show()

This plot is far too crowded to get a real sense of the relationship here. 

A linear regression can help us unpick what the relationship really is

In [ ]:
smf.ols("np.log(sales) ~ np.log(age)", sales).fit().summary().tables[1]

There is a pretty strong positive relationship here that wasn't evident in the scatter plot. BUT it's possible that this relationships depends on whether the firms are online-only or not. We can condition on the other variables and do another regression:

In [ ]:
smf.ols(
    "np.log(sales) ~ np.log(age) + C(online) + products", sales
).fit().summary().tables[1]

The coefficient now looks very different having conditioned on the other confounders. We can see that sales still increase with age, but we don't know if this relationship is linear or not. We could add extra terms (like age squared) to explore this.

Or, we could use a *binned scatterplot* to analyse the relationship non-parametrically. The binned scatterplot divides the conditioning variable, `age` in our example, into equally sized bins or quantiles, and then plots the conditional mean of the dependent variable, `sales` in our example, within each bin.

We'll be using the [**binsreg**](https://nppackages.github.io/binsreg/) package for this {cite:t}`cattaneo2019binscatter`.

Binned scatterplots do not just provide conditional means for optimally chosen intervals; they can also provide inference for these means. In particular, they will give confidence intervals around each data point. In the **binsreg** package, the `ci` option adds confidence intervals to the estimation results.

As the **binsreg** package is not written in the most Pythonic way, we'll adorn it a bit to make it more easily used with dataframes.

In [ ]:
import binsreg


def binscatter(**kwargs):
    # Estimate binsreg
    est = binsreg.binsreg(**kwargs, noplot=True)
    # Retrieve estimates
    df_est = pd.concat([d.dots for d in est.data_plot])
    df_est = df_est.rename(columns={"x": kwargs.get("x"), "fit": kwargs.get("y")})
    # Add confidence intervals
    if "ci" in kwargs:
        df_est = pd.merge(df_est, pd.concat([d.ci for d in est.data_plot]))
        df_est = df_est.drop(columns=["x"])
        df_est["ci"] = df_est["ci_r"] - df_est["ci_l"]
    # Rename groups
    if "by" in kwargs:
        df_est["group"] = df_est["group"].astype(
            kwargs.get("data")[kwargs.get("by")].dtype
        )
        df_est = df_est.rename(columns={"group": kwargs.get("by")})

    return df_est

In [ ]:
# Estimate binsreg
br_est = binscatter(x="age", y="sales", data=sales, ci=(3, 3))
br_est.head()

We can now plot the results

In [ ]:
(
    ggplot(br_est, aes(x="age", y="sales", ymin="ci_l", ymax="ci_r"))
    + geom_point()
    + geom_errorbar()
    + ggtitle("Binscatter: sales as a function of age")
)

We can now see that the relationship is non-linear with a sharp increase in `sales` at the beginning of the lifetime of a firm, followed by a plateau.

As noted, this relationship may be modified by other variables though. So let's now condition on those. 

To condition on `products`, we pass `w=["products"]` to the `binscatter()` function.

In [ ]:
# Estimate binsreg
br_est = binscatter(x="age", y="sales", w=["products"], data=sales, ci=(3, 3))
br_est.head()

Now let's look at the plot of this:

In [ ]:
(
    ggplot(br_est, aes(x="age", y="sales", ymin="ci_l", ymax="ci_r"))
    + geom_point()
    + geom_errorbar()
    + ggtitle("Binscatter: sales as a function of age conditioned on products")
)

Conditional on the number of `products`, the shape of the `sales` life-cycle changes further. Now, after an initial increase in `sales`, we observe a gradual decrease over time.

Do online-only firms have different `sales` life-cycles with respect to mixed online-offline firms? We can produce different binned scatterplots by group using the option `by`.

In [ ]:
# Estimate binsreg
br_est = binscatter(
    x="age", y="sales", w=["products"], by="online", data=sales, ci=(3, 3)
)
br_est["online"] = br_est["online"].astype("boolean")
br_est.head()

In [ ]:
(
    ggplot(br_est, aes(x="age", y="sales", color="online", ymin="ci_l", ymax="ci_r"))
    + geom_point()
    + geom_errorbar()
    + ggtitle("Binscatter: sales as a function of age conditioned on products")
)

From the binned scatterplot, we can see that online products have on average shorter lifetimes, with a higher initial peak in sales, followed by a sharper decline.

Hopefully this shows you how you can use **binsreg** to get a better understanding of causal relationships in your data.

You can find out more about bin scatters in this [video](https://www.youtube.com/watch?v=fg9T2gPZCIs&ab_channel=PaulGoldsmith-Pinkham) by Paul Goldsmith-Pinkham.

## Specification curve analysis

When specifying a model, modellers have many options. These can be informed by field intelligence, priors, and even misguided attempts to find a significant result. Even with the best of intentions, research teams can reach entirely different conclusions using the same, or similar, data because of different choices made in preparing data or in modelling it.

There’s formal evidence that researchers really do make different decisions; one study {cite:ps}`silberzahn2018many` gave the same research question - whether soccer referees are more likely to give red cards to dark-skin-toned players than to light-skin-toned players - to 29 different teams. From the abstract of that paper:

> Analytic approaches varied widely across the teams, and the estimated effect sizes ranged from 0.89 to 2.93 (Mdn = 1.31) in odds-ratio units. Twenty teams (69%) found a statistically significant positive effect, and 9 teams (31%) did not observe a significant relationship. Overall, the 29 different analyses used 21 unique combinations of covariates. Neither analysts’ prior beliefs about the effect of interest nor their level of expertise readily explained the variation in the outcomes of the analyses. Peer ratings of the quality of the analyses also did not account for the variability.

So not only were different decisions made, there seems to be no clearly identifiable reason for them. There is usually scope for reasonable alternative model specifications when estimating coefficients, and those coefficients will vary with those specifications. 

Specification curve analysis {cite:ps}`simonsohn2020specification` looks for a more exhaustive way of trying out alternative specifications. The three steps of specification curve analysis are:

1. identifying the set of theoretically justified, statistically valid, and non-redundant analytic specifications;

2. displaying alternative results graphically, allowing the identification of decisions producing different results; and

3. conducting statistical tests to determine whether as a whole results are inconsistent with the null hypothesis.

For a good example of specification curve analysis in action, see this recent Nature Human Behaviour paper {cite:ps}`orben2019association` on the association between adolescent well-being and the use of digital technology.

We'll use the [**specification curve analysis**](https://aeturrell.github.io/specification_curve) package to do the first two, which you can install with `pip install specification_curve`. To demonstrate the full functionality, we'll use the "mpg" dataset and create a second, alternative 'hp' that is a transformed version of the original.

In [ ]:
mpg = pd.read_csv(
    "https://raw.githubusercontent.com/LOST-STATS/lost-stats.github.io/source/Data/mtcars.csv",
    dtype={"model": str, "mpg": float, "hp": float, "disp": float, "cyl": "category"},
)
mpg.head()

In [ ]:
mpg["hp_boxcox"], _ = st.boxcox(mpg["hp"])
mpg["lnhp"] = np.log(mpg["hp"])

In [ ]:
mpg.info()

Now let's create a specification curve. We need to specify the data, the different outcome variables we'd like to try, `y_endog`; the different possible versions of the main regressor of interest, `x_exog`; the possible controls, `controls`; any controls that should always be included, `always_include`; and any categorical variables to include class-by-class, `cat_expand`. Some of these accept lists of variables as well as single reggressors. The point estimates that have confidence intervals which include zero are coloured in grey, instead of blue. There is also an `exclu_grps` option to exclude certain combinations of regressors, and you can pass alternative estimators to fit, for example `fit(estimator=sm.Logit)`.

In [ ]:
import specification_curve as specy

sc = specy.SpecificationCurve(
    mpg,
    y_endog="mpg",
    x_exog=["lnhp", "hp_boxcox"],
    controls=["drat", "qsec", "cyl", "gear"],
    always_include=["gear"],
    cat_expand="cyl",
)
sc.fit()
sc.plot()